In [ ]:
import os
import requests

def get_repo_name(repo_url):
    return repo_url.rstrip('/').split('/')[-1]

def get_files(repo_url):
    api_url = repo_url.replace("https://github.com/", "https://api.github.com/repos/") + "/contents"
    response = requests.get(api_url)
    response.raise_for_status()
    return response.json()

def download_file(file_url, save_path):
    response = requests.get(file_url)
    response.raise_for_status()
    with open(save_path, 'wb') as file:
        file.write(response.content)

def is_plaintext_file(file_info):
    plain_text_extensions = ('.txt', '.md', '.csv', '.log', '.json', '.yaml', '.yml', '.xml', '.ini', '.conf', '.html', '.rst')
    plain_text_names = ('README', 'LICENSE', 'CONTRIBUTING', 'CHANGES', 'CHANGELOG')
    
    return (file_info['name'].endswith(plain_text_extensions) or
            file_info['name'].split('.')[0] in plain_text_names or
            file_info['name'].lower() in plain_text_names)

def scrape_text_files(repo_url):
    repo_name = get_repo_name(repo_url)
    os.makedirs(repo_name, exist_ok=True)
    try:
        files = get_files(repo_url)
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch files from {repo_url}: {e}")
        return
    
    for file_info in files:
        if file_info['type'] == 'file' and is_plaintext_file(file_info):
            file_url = file_info['download_url']
            save_path = os.path.join(repo_name, file_info['name'])
            try:
                download_file(file_url, save_path)
                print(f"Downloaded {file_info['name']} to {save_path}")
            except requests.exceptions.RequestException as e:
                print(f"Failed to download {file_info['name']} from {file_url}: {e}")

def scrape_repos_from_file(file_path):
    if not os.path.exists(file_path):
        print(f"File {file_path} does not exist.")
        return
    
    with open(file_path, 'r') as file:
        repo_urls = file.readlines()
    
    for repo_url in repo_urls:
        repo_url = repo_url.strip()
        if repo_url:
            print(f"Scraping repository: {repo_url}")
            scrape_text_files(repo_url)

# Example usage
file_path = "repositories.txt"  # Path to your text file containing repository URLs
scrape_repos_from_file(file_path)